<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Software-and-conventions" data-toc-modified-id="Software-and-conventions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Software and conventions</a></span></li></ul></li><li><span><a href="#Required-scripts-and-datasets" data-toc-modified-id="Required-scripts-and-datasets-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Required scripts and datasets</a></span><ul class="toc-item"><li><span><a href="#Regridding-scripts" data-toc-modified-id="Regridding-scripts-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Regridding scripts</a></span></li><li><span><a href="#Data-sources" data-toc-modified-id="Data-sources-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data sources</a></span></li></ul></li><li><span><a href="#Creating-the-directory-structure" data-toc-modified-id="Creating-the-directory-structure-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Creating the directory structure</a></span></li><li><span><a href="#Creating-the-regridding-weight-file" data-toc-modified-id="Creating-the-regridding-weight-file-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Creating the regridding weight file</a></span></li><li><span><a href="#Regridding-the-forcing-data-using-the-weight-file" data-toc-modified-id="Regridding-the-forcing-data-using-the-weight-file-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Regridding the forcing data using the weight file</a></span></li></ul></div>

# Lesson S2 - Regridding GLDAS v2.1 Forcing Data to the WRF-Hydro Domain

## Overview
In this lesson, we will cover regridding GLDAS v2.1 forcing data to a WRF-Hydro domain using the ESMF regridding functions within the NCAR Command Language (NCL).

### Software and conventions
The easiest way to run these lessons is via the [wrfhydro/costarica-training](https://hub.docker.com/r/wrfhydro/costarica-training/) Docker container, which has all software dependencies and data pre-installed.

For a complete description of the software environment used for this training please see [Getting started]().

You may either execute commands by running each cell of this notebook. Alternatively, you may open a terminal in Jupyter Lab by selecting `New -> Terminal` in your `Home` tab of Jupyter Lab and input the commands manually if you prefer. You can also use your own terminal by logging into the container with the command `docker exec -it wrf-hydro-training bash`

All paths used in this lesson assume that the lesson materials are located under your home directory in a folder named `wrf-hydro-training`. If your materials are located in another directory, you will not be able to run the commands in this notebook inside Jupyter and will need to type them manually in your terminal session. 

## Required scripts and datasets

### Regridding scripts
Scripts and documentation to regrid GLDAS v2.1 forcing data can be downloaded from the WRF-Hydro website at https://ral.ucar.edu/projects/wrf_hydro/regridding-scripts.

### Data sources
For this training, test data has already been included. 

**NOTE: These data are a small temporal subset intended to be used for instructional purposes only**

If you would like to acquire other data, GLDAS v2.1 forcing data can be retrieved from a NASA data server at https://hydro1.gesdisc.eosdis.nasa.gov/data/GLDAS/GLDAS_NOAH025_3H.2.1/. 

See the following site for information on how to access data:
https://disc.gsfc.nasa.gov/data-access 

See the following instructions for how to download data files using wget:
https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Download%20Data%20Files%20from%20HTTPS%20Service%20with%20wget

## Creating the directory structure
The NCL regridding scripts require a **very** specific directory structure, which is outlined below:

```
|
input_files
    |_GLDAS_NOAH025_3H.*.nc4 - All GLDAS input forcing data
|_GLDAS2WRFHydro_generate_weights.ncl - NCL script for generating weight file
|_GLDAS2WRFHydro_regrid.ncl - NCL script for regridding
```

In this section, we will create this directory structure

**Step 1: Explore the contents of the GLDAS regridding package provided with this tutorial**
Note: These files are identical to those distributed with the GLDAS regridding package on the WRF-Hydro webpage.  They are unpacked for you in the following directory.

In [ ]:
%%bash
ls ~/wrf-hydro-training/regridding

**Step 2: Create an *input_files* directory to hold the forcing data and move the data to it.**

In [ ]:
%%bash
mkdir -p mkdir ~/wrf-hydro-training/output/lessonS2/input_files
cp ~/wrf-hydro-training/regridding/input_files/*.nc4 \
~/wrf-hydro-training/output/lessonS2/input_files

**Step 3: View the contents of the input_files directory files**

In [ ]:
%%bash
ls ~/wrf-hydro-training/output/lessonS2/input_files | head -10

**Step 4: Copy over the NCL regridding scripts**

In [ ]:
%%bash
cp ~/wrf-hydro-training/regridding/*.ncl \
~/wrf-hydro-training/output/lessonS2/

**Step 5: Check that your directory structure is correct** 

In [ ]:
%%bash
ls ~/wrf-hydro-training/output/lessonS2/

Our directory structure is established, now we can begin the regridding process.

## Creating the regridding weight file
The weight files are netCDF files which specify interpolation weights between the source coordinate data grids and destination coordinate data grids. 

We will use the script *GLDAS2WRFHydro_generate_weights.ncl* to generate the weight file. This script takes a geogrid file and a single GLDAS forcing *.nc4* file as inputs and returns a weight file.

**Step 1: Creating the regridding weight file**
We will use the *GLDAS2WRFHydro_generate_weights.ncl* script to create the weight file

Recall that we created a geogrid file in Lesson S1. We will use this geogrid file as input to the regridding script along with a single GLDAS forcing file. We will run the NCL scripts from our output directory because outputs are written to the current working directory

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lessonS2
ncl 'interp_opt="bilinear"' \
    'srcGridName="./input_files/GLDAS_NOAH025_3H.A20000101.0300.021.nc4"' \
    'dstGridName="~/wrf-hydro-training/example_case/Gridded/DOMAIN/geo_em.d01.nc"' \
    GLDAS2WRFHydro_generate_weights.ncl

In [ ]:
%%bash
ls ~/wrf-hydro-training/output/lessonS2

We have two new files: The *SCRIP_GLDAS_bilinear.nc* describes the GLDAS grids and the *SCRIP_WRFHydro_bilinear.nc* describes the WRF-Hydro grid from the geogrid file.

## Regridding the forcing data using the weight file
In this section we will regrid our forcing data using the weight file generated in the previous section. 

**Step 2: Run the *GLDAS2WRFHydro_regrid.ncl* regridding script**

The *GLDAS2WRFHydro_regrid.ncl* script takes 2 arguments, the path to the geogrid file and the path to GLDAS forcing data. **NOTE: Wildcards can be used in the path to the forcing data to regrid multiple files.**

The *GLDAS2WRFHydro_regrid.ncl* regridding script produces a large amount of standard output so we will pipe the output to a log file.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lessonS2
ncl 'srcFileName="GLDAS_NOAH025_3H.*.nc4"' \
'dstGridName="~/wrf-hydro-training/example_case/Gridded/DOMAIN/geo_em.d01.nc"' \
GLDAS2WRFHydro_regrid.ncl &> regrid.log

**Step 3: View the contents of the *output_files* directory**
The *GLDAS2WRFHydro_regrid.ncl* regridding script will create a directory called *output_files* in the parent directory. Let's view the contents of the directory.

In [ ]:
%%bash
ls ~/wrf-hydro-training/output/lessonS2/output_files

© UCAR 2019